In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from nbdev import *

In [ ]:
import os
import numpy as np
import pandas as pd
import ipywidgets as widgets
from pathlib import Path
from IPython.display import JSON

from tqdm import tqdm

In [ ]:
from ipyannotator.datasets.generators import create_object_detection, xyxy_to_xywh, xywh_to_xyxy

## Select Dataset

In [ ]:
# You can choose between datasets ['CUB_200'] that you can download.
# We use an artifical generated classification dataset by default that doesn't require downloading.

dataset = 'artifical'
# dataset = 'CUB_200'

In [ ]:
if dataset == 'artifical':
    import tempfile
    tmp_dir = tempfile.TemporaryDirectory()
    path = Path(tmp_dir.name)
    print(path)
    
    project_path = path
    project_file = path/'annotations.json'
    image_dir = 'images'
    label_dir = None
    im_width=300 
    im_height=300
    
    from PIL import Image

    create_object_detection(path=path, n_samples=50, n_objects=(1, 1), size=(500, 500))
    annotations = pd.read_json(path/'annotations.json').T

    # Convert artifical dataset annotations to old bbox ipyannotator format
    # {'imagename.jpg': {'x':0, 'y': 0, 'width': 100, 'heigth': 100}}
    anno = annotations.T.to_dict('records')[1]
    annotation_on_explore = {os.path.join(path, image_dir, k): dict(zip(['x', 'y', 'width', 'height'], xyxy_to_xywh(v))) for k, v in anno.items()}

    with open(path/'annotations.json', 'w') as f:  
        json.dump(annotation_on_explore, f)
    
    

## Real DS example

In [ ]:
from ipyannotator.datasets.download import get_cub_200_2011

if dataset == 'CUB_200':
    get_cub_200_2011('data')

    project_path = 'data/CUB_200_2011'
    project_file = 'data/CUB_200_2011/annotaitons_bbox.json'
    image_dir='images'
    label_dir = None
    im_width=50 
    im_height=50
    
    # Let's generate annotattion.json file with bboxes for BBoxAnnotator

    # bboxes
    annotations = pd.read_csv('data/CUB_200_2011/bounding_boxes.txt', delimiter=' ',
                              names='image_id x y width height'.split()).set_index('image_id')

    # images 
    ims = pd.read_csv('data/CUB_200_2011/images.txt', delimiter=' ',
                names='image_id image_name'.split()).set_index('image_id')

    # join by image_id
    anno_ = ims.join(annotations)

    # use full path instead image name
    im_dir = 'data/CUB_200_2011/images/'
    anno_['image_name'] = im_dir + anno_['image_name'].astype(str)

    # combine bbox data columns as single dict to match BBoxAnnotator format
    columns = ['x', 'y', 'width', 'height']
    anno_['bbox'] = anno_[columns].to_dict(orient='records')
    anno_ = anno_.drop(columns=columns)

    # save json {"image_name" : {bbox}, ... }
    annotation_on_explore = anno_.set_index('image_name')['bbox'].to_dict()

    with open('data/CUB_200_2011/annotaitons_bbox.json', 'w') as f:  
        json.dump(annotation_on_explore, f)

# explore

annotated dataset and visualize it to get a feel for it.

- real
- generated
- labeled
- unlabeled

In [ ]:
from ipyannotator.bbox_annotator import BBoxAnnotator

Lets visualize existing annotated dataset.

We use `results_dir` param to indicate directory where `annotation.json` file with existing annotations is located.

You can explore dataset with `next/previous` buttons to check visualized bboxs.

In [ ]:
bb = BBoxAnnotator(project_path=project_path, file_name=project_file, canvas_size=(300,200), image_dir=image_dir)
bb

In [ ]:
# check existing labels 
list(bb.to_dict().items())[10:13]

# create

Load unannotated dataset and create classification labels.

- real
- generated

We set `results_dir="out"` to define that final `annotation.json` file will be saved to `{project_path}/out` direcory. No existing annotations are used now.

In [ ]:
bb = BBoxAnnotator(project_path=project_path, canvas_size=(300, 200), image_dir=image_dir, results_dir='out')
bb

In [ ]:
def augment(sig):
    s = (sig[2] + sig[3]) // 2
    return (sig + (np.random.rand(1, 4) * s - s/2)).astype(int).tolist()[0]

augment(np.array([10,10,10,10]))

In [ ]:
from ipyannotator.datasets.generators import sample_bbox


filt = np.random.uniform(low=0, high=1, size=len(annotation_on_explore))

bbox_noise = 0.1

# lets randomly annotate each image from code and save annotations
for k, f_ in tqdm(zip(bb._model.annotations.keys(), filt)):
    if f_ < bbox_noise:
        bb._model.annotations[k]=dict(zip(['x', 'y', 'width', 'height'], 
                                          augment(np.fromiter(annotation_on_explore[k].values(), dtype=np.uint64))))
    else:
        bb._model.annotations[k] = annotation_on_explore[k]
        
bb._model._update_coords() # update screen

bb._save_btn.click() #save to file


In [ ]:
#same in memory
list(bb.to_dict().items())[10:12]

In [ ]:
bb._model.annotation_file_path

# improve

Load annotated dataset and mark wrongly annotated samples.

- real
- generated

In [ ]:
# open labels generated on [create] step

with open(bb._model.annotation_file_path) as infile:
    annotation_on_create = json.load(infile)

In [ ]:
# back to artificial bbox format ->
# if dataset == 'artifical':
di = {k: xywh_to_xyxy([v['x'], v['y'], v['width'], v['height']]) if v else [] for k, v in annotation_on_create.items()}
list(di.items())[:5]

In [ ]:
# lets save drawn bboxes to real img files

from ipyannotator.datasets.generators import draw_bbox
from PIL import Image, ImageDraw
from tqdm import tqdm

captured_path = Path(project_path) / "captured"

for im, bbx in tqdm(di.items()):
    # use captured_path instead image_dir, keeping the folder structure
    old_im_path = Path(im)
    index = old_im_path.parts.index(image_dir)+1
    new_im_path = captured_path.joinpath(*old_im_path.parts[index:])
    new_im_path.parent.mkdir(parents=True, exist_ok=True)
    
    _image = Image.open(im)
    if bbx:
        try:           
            # todo: debug draw_bbox: function takes exactly 1 argument (3 given)
            _image = draw_bbox(bbx, im=_image, black=False, values=False, width=3+np.prod(_image.size)//50000)
        except Exception as e: 
            pass 
    _image.save(new_im_path)
    

You should __mark all errors__ (images, with bboxs that aren't closely around an object)

In [ ]:
from ipyannotator.capture_annotator import CaptureAnnotator

c = CaptureAnnotator(project_path, image_dir='captured', 
                     image_width=150, image_height=150, 
                     n_rows=2, n_cols=4,
                     question=f'Check images with incorrect or empty bbox annotation', 
                     results_dir=f'missed')
c

In [ ]:
# mark incorrect bboxes from code

im_dir_path = Path(project_path) / image_dir

for k, v in tqdm(c._model.annotations.items()):
    capture_im_path = Path(k)
    
    index = capture_im_path.parts.index('captured') + 1
    new_im_path = im_dir_path.joinpath(*capture_im_path.parts[index:])
    
    v_expl =annotation_on_explore.get(str(new_im_path), {})
    v_cret =annotation_on_create.get(str(new_im_path), {})
    
    c._model.annotations[k] = {'answer': v_expl != v_cret}   
    c._model._update_state()
    
c._save_btn.click()
# print(c._model.annotations)

Now we can get list of all images which incorrect bounding boxes.

Let's get list of images with absent or incorrect annotations marked on previous above

In [ ]:
[k for k, v in c.to_dict().items() if v['answer']][:15]